## 1-5 fine_tuning 으로 정확도를 향상시키는 방법

- 학습된 VGG 모델을 사용해서 fine_tuning으로 개미와 벌을 이미지 분류하는 모델을 학습시키기

### 사전준비

In [35]:
# 패키지 Import 
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import models
from tqdm import tqdm

In [36]:
# 난수 시드 설정
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

## Dataset과 DataLoader 작성

In [37]:
#import sys
# sys.path.append('C:\pythonStudy\Pytorch_Flipped_Learning\CH1_image_classification')
from utils.dataloader_image_classification import ImageTransform, make_datapath_list, HymenopteraDataset

# 거미와 벌의 이미지 파일 경로 리스트 작성
train_list = make_datapath_list(phase='train')
val_list = make_datapath_list(phase='val')

size = 2242
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

train_dataset = HymenopteraDataset(
    file_list = train_list, transform=ImageTransform(size, mean, std), phase='train'
    )

val_dataset = HymenopteraDataset(
    file_list = val_list, transform = ImageTransform(size, mean, std), phase='val'
)

batch_size = 4

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False
)

dataloaders_dict = {'train': train_dataloader , 'val':val_dataloader}

./data/hymenoptera_data/train/**/*.jpg
./data/hymenoptera_data/val/**/*.jpg


## 네트워크 모델 작성하기

In [38]:
# 학습된 VGG-16 모델 로드하기
use_pretrained = True
net = models.vgg16(pretrained=use_pretrained)

#vgg16 의 마지막 출력층 유닛을 개마, 벌 총 2개로 바꿔주기
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# trian모드로 설정
net.train()

print('네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다')

네트워크 설정 완료: 학습된 가중치를 로드하고 훈련 모드로 설정했습니다


## 손실함수 정의

In [39]:
# 손실 함수의 설정
criterion = nn.CrossEntropyLoss()

## 최적화 방법 설정

In [40]:
## 모든 층의 파라미터를 학습할 수 있도록 optimizer 를 설정

## fine tuning 으로 학습할 파라미터를 변수에 저장하기
params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

## 학습시킬 층의 파라미터명을 지정
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

## 파라미터를 각각 리스트에 저장
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1에 저장: ", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2에 저장: ", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3에 저장: ", name)

    else:
        param.requires_grad = False
        print("경사 계산없음. 학습하지 않음: ", name)

params_to_update_1에 저장:  features.0.weight
params_to_update_1에 저장:  features.0.bias
params_to_update_1에 저장:  features.2.weight
params_to_update_1에 저장:  features.2.bias
params_to_update_1에 저장:  features.5.weight
params_to_update_1에 저장:  features.5.bias
params_to_update_1에 저장:  features.7.weight
params_to_update_1에 저장:  features.7.bias
params_to_update_1에 저장:  features.10.weight
params_to_update_1에 저장:  features.10.bias
params_to_update_1에 저장:  features.12.weight
params_to_update_1에 저장:  features.12.bias
params_to_update_1에 저장:  features.14.weight
params_to_update_1에 저장:  features.14.bias
params_to_update_1에 저장:  features.17.weight
params_to_update_1에 저장:  features.17.bias
params_to_update_1에 저장:  features.19.weight
params_to_update_1에 저장:  features.19.bias
params_to_update_1에 저장:  features.21.weight
params_to_update_1에 저장:  features.21.bias
params_to_update_1에 저장:  features.24.weight
params_to_update_1에 저장:  features.24.bias
params_to_update_1에 저장:  features.26.weight
params_to_update_1

In [41]:
# # fine tuning 으로 학습할 파라미터를 변수에 저장하기 
# # 
# params_to_update_1 = []
# params_to_update_2 = []
# params_to_update_3 = []

# # 학습시킬 층의 파라미터명을 지정
# update_param_names_1 = ['features']
# update_param_names_2 = ['classifier.0.weight',
#                         'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias']
# update_param_names_3 = ['classifier.6.weight', 'classifier.6.bias']

# # 파라미터를 각각 리스트에 저장
# for name, param in net.named_parameters():
#     if update_param_names_1[0] in name:
#         param.requires_grad = True
#         params_to_update_1.append(param)
#         print('parmas_to_update_1에 저장 : ', name)

#     elif name in update_param_names_2:
#         param.requires_grad = True
#         params_to_update_2.append(param)
#         print('params_to_update2에 저장 : ', name)

#     elif name in update_param_names_3:
#         param.requires_grad = True
#         params_to_update_3.append(param)
#         print('params_to_update_3에 저장 : ',name)

#     else:
#         param.requires_grad = False
#         print('경사 계산 없음. 학습하지 않음 : ', name)

> 지수 표기법 : https://izen8.tistory.com/902

In [42]:
# # 각 층마다 다른 lr를 적용할 수 있다.
# # 최적화 기법 설정
# optimizer = optim.SGD([
#     {'params' : params_to_update_1, 'lr': 1e-4}, # 0.0001
#     {'params' : params_to_update_2, 'lr': 5e-4}, # 0.0005
#     {'params' : params_to_update_3, 'lr': 1e-3} # 0.001
    
# ], momentum=0.9)


In [43]:
## 각각 층마다 다른 lr 를 적용할 수 있다.

# 최적화 기법 설정
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},  ## 0.0001
    {'params': params_to_update_2, 'lr': 5e-4},  ## 0.0005
    {'params': params_to_update_3, 'lr': 1e-3}   ## 0.001
], momentum=0.9)

In [44]:
print(optimizer)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.0001
    momentum: 0.9
    nesterov: False
    weight_decay: 0

Parameter Group 1
    dampening: 0
    lr: 0.0005
    momentum: 0.9
    nesterov: False
    weight_decay: 0

Parameter Group 2
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)


## 학습 및 검증 실시

In [45]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('사용 장치 : ', device)

사용 장치 :  cpu


In [46]:
net.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [47]:
# 모델을 학습시키는 ㅎ마수
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 초기설정
    # GPU가 사용가능한지 확인함
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print('사용 장치 : ', device)

    # 네트워크를 직접 사용하고 있는 디바이스
    net.to(device)

    # 네트워크가 어느정도 고정됨녀 가속화
    torch.backends.cudnn.benchmark = True

    # epoch
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------------------------')

        # epoch 별 훈련 및 검증
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train() # train mode

            else:
                net.eval()

            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch ==0) and(phase == 'train'):
                continue

            for inputs, labels in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):

                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()


                    epoch_loss += loss.item() * inputs.size(0)

                    epoch_corrects += torch.sum(preds == labels.data)
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double()/len(dataloaders_dict[phase].dataset)

            print('{} loss : {:.4f} Acc : {:.4f}'.format(phase, epoch_loss, epoch_acc))


In [48]:
# # 모델을 학습시키는 함수작성
# def train_model(net, dataloaders_dict, criterion, oprimizer, num_epochs):

#     #초기 성정
#     #GPU가 사용 가능한지 확인
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     print("사용 장치: ", device)

#     # 네트워크를 GPU로
#     net.to(device)

#     # 네트워크가 어느정도 고정되면 고속화시킨다.
#     torch.backends.cudnn.benchmark = True

#     # 에폭 루프
#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch+1, num_epochs))
#         print('------------')

#         # 에폭별 훈련 및 검증 루프
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 net.train() # 모델을 훈련모드로
#             else:
#                 net.eval() # 모델을 검증모드로

#             epoch_loss = 0.0 # 에폭 손실 함
#             epoch_corrects = 0 # 에폭 정담 수

#             # 학습하지 않았을 때의 검증 성능을 확인 하기위해 epoch=0의 훈련 생략
#             if (epoch == 0) and (phase == 'train'):
#                 continue

#             # 데이터 로더에서 미니배치를 꺼내 루프
#             for inputs, labels in tqdm(dataloaders_dict[phase]):
#                 #GPU를 사용할 수 있다면 GPU에 데이터를 보낸다.
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 #옵티마이저 초기화
#                 optimizer.zero.grad()

#                 #순전파 계산
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = net(inputs)
#                     loss = criterion(outputs, labels) # 손실계산
#                     _, preds = torch.max(outputs, 1) # 라벨 예측

#                 # 훈련 시에는 오차 역전파법
#                 if phase == 'train':
#                     loss.backward()
#                     optimizer.step()

#                 #결과 계산
#                 epoch_loss += loss.item() * inputs.size(0) # 손실의 합계 갱신
#                 # 정답 수의 합계 갱신
#                 epoch_corrects += torch.sum(preds == labels.data)
#             #에폭별 손실과 정답률 표시
#             epoch_loss = epoch_loss /len(dataloaders_dict[phase].dataset)
#             epoch_acc  = epoch_corrects.double(
#             ) / len(dataloaders_dict[phase].dataset)

#             print('{} Loss: {:,4f} Acc: {:,4f}'.format(phase, epoch_loss, epoch_acc))

In [49]:
# # 모델을 학습시키는 함수를 작성
# def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

#     # 초기 설정
#     # GPU가 사용 가능한지 확인
#     device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#     print("사용 장치: ", device)

#     # 네트워크를 GPU로
#     net.to(device)

#     # 네트워크가 어느 정도 고정되면, 고속화시킨다
#     torch.backends.cudnn.benchmark = True

#     # epoch 루프
#     for epoch in range(num_epochs):
#         print('Epoch {}/{}'.format(epoch+1, num_epochs))
#         print('-------------')

#         # epoch별 훈련 및 검증 루프
#         for phase in ['train', 'val']:
#             if phase == 'train':
#                 net.train()  # 모델을 훈련 모드로
#             else:
#                 net.eval()   # 모델을 검증 모드로

#             epoch_loss = 0.0  # epoch 손실의 합
#             epoch_corrects = 0  # epoch 정답수

#             # 미학습시의 검증 성능을 확인하기 위해 epoch=0의 훈련은 생략
#             if (epoch == 0) and (phase == 'train'):
#                 continue

#             # 데이터 로더에서 미니 배치를 꺼내 루프
#             for inputs, labels in tqdm(dataloaders_dict[phase]):

#                 # GPU가 사용 가능하면 GPU에 데이터 보내기
#                 inputs = inputs.to(device)
#                 labels = labels.to(device)

#                 # optimizer를 초기화
#                 optimizer.zero_grad()

#                 # 순전파(forward) 계산
#                 with torch.set_grad_enabled(phase == 'train'):
#                     outputs = net(inputs)
#                     loss = criterion(outputs, labels)  # 손실 계산
#                     _, preds = torch.max(outputs, 1)  # 라벨 예측

#                     # 훈련시에는 오차 역전파법
#                     if phase == 'train':
#                         loss.backward()
#                         optimizer.step()

#                     # 결과 계산
#                     epoch_loss += loss.item() * inputs.size(0)  # loss의 합계를 갱신
#                     # 정답 수의 합계를 갱신
#                     epoch_corrects += torch.sum(preds == labels.data)

#             # epoch별 loss와 정답률을 표시
#             epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
#             epoch_acc = epoch_corrects.double(
#             ) / len(dataloaders_dict[phase].dataset)

#             print('{} Loss: {:.4f} Acc: {:.4f}'.format(
#                 phase, epoch_loss, epoch_acc))

In [50]:
## 학습 및 검증 실행
num_epochs=2
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

사용 장치 :  cpu
Epoch 1/2
-------------------------------


 38%|███▊      | 15/39 [33:19<51:37, 129.07s/it]  

- 계속된 에러 해결책 : https://mingchin.tistory.com/419
- batch_size 문제

In [ ]:
##